# Cyberbullying Detection
Proses dibagi menjadi

1.   Preprocessing 
2.   Feature Extraction
3.   Classification
4.   Evaluation



#Data preparation


Import Package yang dibutuhkan 

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords #import stopwords
from nltk.tokenize import RegexpTokenizer
import re, string, unicodedata #import regular expression
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import pandas as pd
import csv

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def casefolding(str):
    new_str = str.lower()
    return new_str

def cleaning(str):
    #melakukan masking terhadap url sehingga menjadi _url_
    str =  re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}     /)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    str =str.replace('<br>','')
    str = str.replace('</br>','')
    str = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str)
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
    #remove digit from string
    str = re.sub("\S*\d\S*", "", str).strip()
    #removeHashtag
    str = re.sub('#[^\s]+','',str)
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #to lowercase
    str = str.lower()
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
    return str


In [ ]:
def stopword(str):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(str) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]  
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    return(filtered_sentence)

In [ ]:
#remove sentence which contains only one word
def removeSentence(str): 
    word = str.split()
    wordCount = len(word)
    if(wordCount<=1):
        str = ''
    
    return str

In [ ]:
def preprocessing (str):
    str1 = casefolding(str)
    str2 = cleaning(str1)
    str3 = stopword(str2)
    str4 = ' '.join(str3)
    str5 = removeSentence(str4)
    
    return str5

In [ ]:
coba = ('@halalflaws @biebervalue @greenlinerzjm I read them in context.No change in meaning. The history of Islamic slavery. https://t.co/xWJzpSodGj')
pre = preprocessing(coba)
print(pre)

read context change meaning history islamic slavery


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
fo = pd.read_csv('/content/gdrive/MyDrive/TUGAS NLP/twitter_parsed_dataset.csv')
fo.Text=fo.Text.astype(str)
review = fo['Text'].tolist()
annotation = fo['Annotation'].tolist()
def process(kata):
    rv = []
    for p in kata:
        str = preprocessing(p)
        rv.append(str)
    return(rv)
process = process(review)

In [ ]:
bullying_textpreprocessing = {'Text':process,
                              'Annotation' : annotation}
df = pd.DataFrame(bullying_textpreprocessing )
df['Annotation'] = df['Annotation'].replace(['none','sexism','racism'],[1, 2, 3])
display(df)

,Text,Annotation
0,read context change meaning history islamic sl...,1.0
1,idiots claim people tried stop becoming terror...,1.0
2,rt call sexist go auto place rather talk guy,2.0
3,wrong isis follows example mohammed quran exactly,3.0
4,,1.0
...,...,...
16846,feeling sorry girls safe kat andre going home mkr,1.0
16847,mkr pretty good dishes happy ok well never eat...,1.0
16848,rt deconstructed lemon tart please go one seas...,1.0
16849,stupid talk blocked,1.0


# Feature Extraction


*   TF-IDF


In [ ]:
import numpy as np
from imblearn.over_sampling import SMOTE
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
df.replace('', np.nan, inplace=True)
df.dropna(inplace=True)
none = df.loc[df['Annotation'] == 1, 'Text'].copy().reset_index(drop=True)
sexism = df.loc[df['Annotation'] == 2, 'Text'].copy().reset_index(drop=True)
racism = df.loc[df['Annotation'] == 3, 'Text'].copy().reset_index(drop=True)

In [ ]:
len(none), len(sexism), len(racism)

(10821, 3313, 1968)

In [ ]:
y = df['Annotation']
y=y.to_frame()

In [ ]:
vect = TfidfVectorizer(max_features=15000, min_df=5, max_df=0.7)
vector_transformer = vect.fit(df['Text'])
vectorized_df = vector_transformer.transform(df['Text'])

In [ ]:
smote = SMOTE(random_state=777,k_neighbors=5)
X_smote,y_smote = smote.fit_sample(vectorized_df,y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_trainSMOTE, X_valSMOTE, y_trainSMOTE, y_valSMOTE = train_test_split(X_smote, y_smote, test_size=0.2, random_state=0)
X_trainSMOTE, X_testSMOTE, y_trainSMOTE, y_testSMOTE = train_test_split(X_trainSMOTE, y_trainSMOTE, test_size=0.25, random_state=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vectorized_df, df['Annotation'].values, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_val.shape, X_test.shape

((9660, 3855), (3221, 3855), (3221, 3855))

In [ ]:
X_trainSMOTE.shape, X_valSMOTE.shape, X_testSMOTE.shape

((19477, 3855), (6493, 3855), (6493, 3855))

# Classification


*   Deep Learning
*   ML biasa (dipilih): Multinomial Naive Bayes, Linear Support Vector Classification (SVC) dan Logistic Regression (LR)



In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import sklearn.svm as svm
from nltk.tokenize import sent_tokenize 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pickle

#ML w/o SMOTE

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
predval_nb = nb.predict(X_val)
print('Accuracy %s' % accuracy_score(predval_nb,y_val))
print(classification_report(y_val,predval_nb))

Accuracy 0.7929214529649177
              precision    recall  f1-score   support

         1.0       0.78      0.96      0.86      2186
         2.0       0.88      0.38      0.53       664
         3.0       0.81      0.53      0.64       371

    accuracy                           0.79      3221
   macro avg       0.82      0.62      0.68      3221
weighted avg       0.80      0.79      0.77      3221



In [ ]:
pred_nb = nb.predict(X_test)
print('Accuracy %s' % accuracy_score(pred_nb,y_test))
print(classification_report(y_test, pred_nb))

Accuracy 0.7926109903756597
              precision    recall  f1-score   support

         1.0       0.78      0.96      0.86      2156
         2.0       0.90      0.39      0.55       672
         3.0       0.78      0.58      0.66       393

    accuracy                           0.79      3221
   macro avg       0.82      0.64      0.69      3221
weighted avg       0.81      0.79      0.77      3221



In [ ]:
filenameNB = 'Multinomial NB without SMOTE.sav'
pickle.dump(nb, open(filenameNB, 'wb'))

In [ ]:
clf_svm = svm.LinearSVC()
clf_svm.fit(X_trainSMOTE, y_trainSMOTE)
pred_svm = clf_svm.predict(X_valSMOTE)
print('Accuracy %s' % accuracy_score(pred_svm,y_valSMOTE))
print(classification_report(y_valSMOTE,pred_svm))

Accuracy 0.8851070383489912
              precision    recall  f1-score   support

         1.0       0.87      0.77      0.82      2130
         2.0       0.85      0.90      0.88      2230
         3.0       0.93      0.99      0.96      2133

    accuracy                           0.89      6493
   macro avg       0.88      0.88      0.88      6493
weighted avg       0.88      0.89      0.88      6493



In [ ]:
clf_svm = svm.LinearSVC()
clf_svm.fit(X_train, y_train)
pred_svm = clf_svm.predict(X_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.8149642968022354
              precision    recall  f1-score   support

         1.0       0.83      0.91      0.87      2186
         2.0       0.78      0.58      0.66       664
         3.0       0.74      0.66      0.70       371

    accuracy                           0.81      3221
   macro avg       0.79      0.72      0.75      3221
weighted avg       0.81      0.81      0.81      3221



In [ ]:
predtest_svm = clf_svm.predict(X_test)
print('Accuracy %s' % accuracy_score(predtest_svm,y_test))
print(classification_report(y_test,predtest_svm))

Accuracy 0.8096864327848494
              precision    recall  f1-score   support

         1.0       0.83      0.90      0.86      2156
         2.0       0.76      0.59      0.67       672
         3.0       0.74      0.69      0.71       393

    accuracy                           0.81      3221
   macro avg       0.78      0.73      0.75      3221
weighted avg       0.81      0.81      0.80      3221



In [ ]:
filenameSVM = 'SVM.sav'
pickle.dump(clf_svm, open(filenameSVM, 'wb'))

In [ ]:
logReg = LogisticRegression(C=0.2, dual=False)
logReg.fit(X_train, y_train)
predval_logreg = logReg.predict(X_val)
print('Accuracy %s' % accuracy_score(predval_logreg,y_val))
print(classification_report(y_val,predval_logreg))

Accuracy 0.778329711269792
              precision    recall  f1-score   support

         1.0       0.77      0.97      0.86      2186
         2.0       0.91      0.36      0.52       664
         3.0       0.78      0.39      0.52       371

    accuracy                           0.78      3221
   macro avg       0.82      0.57      0.63      3221
weighted avg       0.80      0.78      0.75      3221



In [ ]:
predtest_logreg = logReg.predict(X_test)
print('Accuracy %s' % accuracy_score(predtest_logreg,y_test))
print(classification_report(y_test,predtest_logreg))

Accuracy 0.7755355479664701
              precision    recall  f1-score   support

         1.0       0.76      0.97      0.85      2156
         2.0       0.89      0.38      0.53       672
         3.0       0.79      0.41      0.54       393

    accuracy                           0.78      3221
   macro avg       0.81      0.58      0.64      3221
weighted avg       0.79      0.78      0.75      3221



In [ ]:
filenameLogReg = 'Logostic Regression.sav'
pickle.dump(logReg, open(filenameLogReg, 'wb'))

In [ ]:
def predictor(sentence):
    sent = sent_tokenize(sentence)
    y = vector_transformer.transform(sent)
 #   y_pred = nb.predict(y)
    loaded_model = pickle.load(open(filenameSVM, 'rb'))
    y_pred = loaded_model.predict(y)
    if y_pred == 1:
      return ('none')  

    elif y_pred  == 2:
        return('sexism')

    elif y_pred == 3:
        return('racism')

stc = input('Enter tweet : ')
print(predictor(stc))

Enter tweet : i hate muslim
racism


#ML SMOTE

In [ ]:
nbSMOTE = MultinomialNB()
nbSMOTE.fit(X_trainSMOTE, y_trainSMOTE)
predvalSMOTE_nb = nbSMOTE.predict(X_valSMOTE)
print('Accuracy %s' % accuracy_score(predvalSMOTE_nb,y_valSMOTE))
print(classification_report(y_valSMOTE,predvalSMOTE_nb))

Accuracy 0.8295087016787309
              precision    recall  f1-score   support

         1.0       0.79      0.70      0.74      2130
         2.0       0.83      0.82      0.82      2230
         3.0       0.87      0.97      0.92      2133

    accuracy                           0.83      6493
   macro avg       0.83      0.83      0.83      6493
weighted avg       0.83      0.83      0.83      6493



In [ ]:
pred_nbSMOTE = nbSMOTE.predict(X_testSMOTE)
print('Accuracy %s' % accuracy_score(pred_nbSMOTE,y_testSMOTE))
print(classification_report(y_testSMOTE,pred_nbSMOTE))

Accuracy 0.8348991221315263
              precision    recall  f1-score   support

         1.0       0.79      0.71      0.75      2119
         2.0       0.83      0.82      0.83      2203
         3.0       0.87      0.97      0.92      2171

    accuracy                           0.83      6493
   macro avg       0.83      0.83      0.83      6493
weighted avg       0.83      0.83      0.83      6493



In [ ]:
filenameNBSMOTE = 'Multinomial NB with SMOTE.sav'
pickle.dump(nbSMOTE, open(filenameNBSMOTE, 'wb'))

In [ ]:
clf_svmSMOTE = svm.LinearSVC()
clf_svmSMOTE.fit(X_trainSMOTE, y_trainSMOTE)
predval_svmSMOTE = clf_svmSMOTE.predict(X_valSMOTE)
print('Accuracy %s' % accuracy_score(predval_svmSMOTE,y_valSMOTE))
print(classification_report(y_valSMOTE,predval_svmSMOTE))

Accuracy 0.8851070383489912
              precision    recall  f1-score   support

         1.0       0.87      0.77      0.82      2130
         2.0       0.85      0.90      0.88      2230
         3.0       0.93      0.99      0.96      2133

    accuracy                           0.89      6493
   macro avg       0.88      0.88      0.88      6493
weighted avg       0.88      0.89      0.88      6493



In [ ]:
predtest_svmSMOTE = clf_svmSMOTE.predict(X_testSMOTE)
print('Accuracy %s' % accuracy_score(predtest_svmSMOTE,y_testSMOTE))
print(classification_report(y_testSMOTE,predtest_svmSMOTE))

Accuracy 0.8807947019867549
              precision    recall  f1-score   support

         1.0       0.86      0.76      0.81      2119
         2.0       0.85      0.90      0.87      2203
         3.0       0.93      0.97      0.95      2171

    accuracy                           0.88      6493
   macro avg       0.88      0.88      0.88      6493
weighted avg       0.88      0.88      0.88      6493



In [ ]:
filenameSVMSMOTE = 'SVM with SMOTE.sav'
pickle.dump(clf_svmSMOTE, open(filenameSVMSMOTE, 'wb'))

In [ ]:
logReg = LogisticRegression(C=0.2, dual=False)
logReg.fit(X_trainSMOTE, y_trainSMOTE)
predval_logreg = logReg.predict(X_valSMOTE)
print('Accuracy %s' % accuracy_score(predval_logreg,y_valSMOTE))
print(classification_report(y_valSMOTE,predval_logreg))

Accuracy 0.8600030802402587
              precision    recall  f1-score   support

         1.0       0.80      0.79      0.79      2130
         2.0       0.85      0.86      0.85      2230
         3.0       0.94      0.93      0.93      2133

    accuracy                           0.86      6493
   macro avg       0.86      0.86      0.86      6493
weighted avg       0.86      0.86      0.86      6493



In [ ]:
pred_logreg = logReg.predict(X_testSMOTE)
print('Accuracy %s' % accuracy_score(pred_logreg,y_testSMOTE))
print(classification_report(y_testSMOTE,pred_logreg))

Accuracy 0.8529185276451563
              precision    recall  f1-score   support

         1.0       0.78      0.78      0.78      2119
         2.0       0.84      0.85      0.85      2203
         3.0       0.93      0.93      0.93      2171

    accuracy                           0.85      6493
   macro avg       0.85      0.85      0.85      6493
weighted avg       0.85      0.85      0.85      6493



In [ ]:
filenamelogReg = 'Logistic Regression with SMOTE.sav'
pickle.dump(logReg, open(filenamelogReg, 'wb'))

In [ ]:
def predictor_SMOTE(sentence):
    sent = sent_tokenize(sentence)
    y = vector_transformer.transform(sent)
 #   y_pred = nb.predict(y)
    loaded_model = pickle.load(open(filenameSVMSMOTE, 'rb'))
    y_pred = loaded_model.predict(y)
    if y_pred == 1:
      return ('none')  

    elif y_pred  == 2:
        return('sexism')

    elif y_pred == 3:
        return('racism')

stc = input('Enter tweet : ')
print(predictor_SMOTE(stc))

Enter tweet : i hate muslim
racism


Deep Learning w/o SMOTE

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import regularizers
from keras.layers import LSTM, Conv1D, MaxPooling1D, Activation, Dense, Dropout, Input, Embedding, Bidirectional, GlobalMaxPool1D

In [ ]:
# 3 news groups
num_labels = 3
vocab_size = 15000
batch_size = 100
num_epochs = 10

In [ ]:
train_size = int(len(df) * .6)
test_size = int(len(df)* .2)
val_size = int(len(df)* .2)
train_size, test_size, val_size

(9661, 3220, 3220)

In [ ]:
train_posts = df['Text'][:train_size]
train_tags = df['Annotation'][:train_size]

test_posts = df['Text'][:test_size]
test_tags = df['Annotation'][:test_size]

val_posts = df['Text'][:val_size]
val_tags = df['Annotation'][:val_size]

In [ ]:
type(test_posts)

pandas.core.series.Series

In [ ]:
len(train_tags), type(test_tags), len(val_tags)

(9661, pandas.core.series.Series, 3220)

In [ ]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(train_posts) 

# pad sequences
max_length = max([len(s.split()) for s in train_posts])


# define vocabulary size
vocab_size = len(tokenizer_obj.word_index) + 1

X_train_tokens =  tokenizer_obj.texts_to_sequences(train_posts)
X_test_tokens = tokenizer_obj.texts_to_sequences(test_posts)
X_val_tokens = tokenizer_obj.texts_to_sequences(val_posts)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')
X_val_pad = pad_sequences(X_val_tokens, maxlen=max_length, padding='post')

encoder = LabelBinarizer()
encoder.fit(train_tags)
y_trainDL = encoder.transform(train_tags)
y_testDL = encoder.transform(test_tags)
y_valDL = encoder.transform(val_tags)

In [ ]:
type(X_train_tokens)

list

In [ ]:
X_train_pad.shape, y_trainDL.shape

((9661, 21), (9661, 3))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

EMBEDDING_DIM = 100
def build_model(vocab_size, EMBEDDING_DIM, max_length):
  print('Build model...')

  model = Sequential()
  model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
  model.add(GRU(units=32,  dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(num_labels, activation='softmax'))

  # try using different optimizers and different optimizer configs
  
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
print('Summary of the built model...')
model = build_model(vocab_size=vocab_size, EMBEDDING_DIM=EMBEDDING_DIM, max_length=max_length)
print(model.summary())

Summary of the built model...
Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 21, 100)           1239400   
_________________________________________________________________
gru (GRU)                    (None, 32)                12864     
_________________________________________________________________
dense (Dense)                (None, 3)                 99        
Total params: 1,252,363
Trainable params: 1,252,363
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
num_epochs =10
batch_size = 128
history = model.fit(X_train_pad, y_trainDL,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=2,
                    validation_data = [X_val_pad, y_valDL])

Epoch 1/10
76/76 - 14s - loss: 0.8800 - accuracy: 0.6687 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
76/76 - 9s - loss: 0.8327 - accuracy: 0.6706 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
76/76 - 9s - loss: 0.5821 - accuracy: 0.7480 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
76/76 - 9s - loss: 0.4149 - accuracy: 0.8465 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
76/76 - 9s - loss: 0.2267 - accuracy: 0.9255 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
76/76 - 9s - loss: 0.1509 - accuracy: 0.9509 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
76/76 - 9s - loss: 0.1121 - accuracy: 0.9660 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 8/10
76/76 - 9s - loss: 0.0950 - accuracy: 0.9705 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 9/10
76/76 - 9s - loss: 0.0752 - accuracy: 0.9773 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 10/10
76/76 - 9s - loss: 0.0628 - accur

In [ ]:
score,acc= model.evaluate(X_test_pad, y_testDL, 
                          batch_size = batch_size, verbose = 2)


26/26 - 1s - loss: 0.0340 - accuracy: 0.9907


In [ ]:
type(X_test_pad)

numpy.ndarray

In [ ]:
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
import tensorflow as tf


In [ ]:
def predict_DL(s):
  s = sent_tokenize(s)
  text_labels = encoder.classes_
  X_tokens = tokenizer_obj.texts_to_sequences(s)
  X_pad = pad_sequences(X_tokens, maxlen=max_length, padding='post')
  loaded_model= tf.keras.models.load_model('model.h5')
  predictions = model.predict(np.array(X_pad))
  predicted_label = text_labels[np.argmax(predictions[0])]
  if predicted_label == 1.0:
    return ('none')  

  elif predicted_label  == 2.0:
      return('sexism')

  elif predicted_label == 3.0:
      return('racism')

s = input('Enter tweet : ')
print(predict_DL(s))

Enter tweet : i hate muslim
racism


Deep Learning SMOTE

In [ ]:
smote = SMOTE('minority')
X_sm, y_sm = smote.fit_sample(X_train_pad, y_trainDL)
print(X_sm.shape, y_sm.shape)

(14940, 21) (14940, 3)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
y_train_labels = np.array(train_tags)
np.unique(y_train_labels)

array([1., 2., 3.])

In [ ]:
y_test_labels = np.array(test_tags)
np.unique(y_test_labels)

array([1., 2., 3.])

In [ ]:
modelSMOTE = build_model(vocab_size=vocab_size, EMBEDDING_DIM=EMBEDDING_DIM, max_length=max_length)
print(modelSMOTE.summary())

Build model...
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 21, 100)           1239400   
_________________________________________________________________
gru_2 (GRU)                  (None, 32)                12864     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 99        
Total params: 1,252,363
Trainable params: 1,252,363
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_sample_weight("balanced", np.unique(y_train_labels))
num_epochs =10
batch_size = 128
history_SMOTE = modelSMOTE.fit(X_sm, y_sm,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=2,
                    #class_weight=class_weight,
                    validation_data = [X_val_pad, y_testDL])

Epoch 1/10
117/117 - 16s - loss: 1.0000 - accuracy: 0.4321 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
117/117 - 13s - loss: 0.9626 - accuracy: 0.4699 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
117/117 - 13s - loss: 0.6624 - accuracy: 0.7062 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
117/117 - 13s - loss: 0.4590 - accuracy: 0.8258 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
117/117 - 13s - loss: 0.3168 - accuracy: 0.8912 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
117/117 - 13s - loss: 0.2328 - accuracy: 0.9235 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
117/117 - 13s - loss: 0.1781 - accuracy: 0.9437 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 8/10
117/117 - 13s - loss: 0.1418 - accuracy: 0.9549 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 9/10
117/117 - 13s - loss: 0.1111 - accuracy: 0.9655 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 10/10
117/117

In [ ]:
score1,acc1= modelSMOTE.evaluate(X_test_pad, y_testDL, 
                          batch_size = batch_size, verbose = 2)

26/26 - 1s - loss: 0.0557 - accuracy: 0.9839


In [ ]:
modelSMOTE.save("modelSMOTE.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
def predictSMOTE_DL(s):
  s = sent_tokenize(s)
  text_labels = encoder.classes_
  X_tokens = tokenizer_obj.texts_to_sequences(s)
  X_pad = pad_sequences(X_tokens, maxlen=max_length, padding='post')
  loaded_model= tf.keras.models.load_model('modelSMOTE.h5')
  predictions = model.predict(np.array(X_pad))
  predicted_label = text_labels[np.argmax(predictions[0])]
  if predicted_label == 1.0:
    return ('none')  

  elif predicted_label  == 2.0:
      return('sexism')

  elif predicted_label == 3.0:
      return('racism')

s = input('Enter tweet : ')
print(predictSMOTE_DL(s))

Enter tweet : i hate muslim
racism


Deep Learning LSTM w/o SMOTE


In [ ]:
def build_modelLSTM(vocab_size, EMBEDDING_DIM, max_length):
  print('Build model...')

  model = Sequential()
  model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
  model.add(LSTM(units=32,  dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(num_labels, activation='softmax'))

  # try using different optimizers and different optimizer configs
  
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
print('Summary of the built model...')
modelLSTM = build_modelLSTM(vocab_size=vocab_size, EMBEDDING_DIM=EMBEDDING_DIM, max_length=max_length)
print(modelLSTM.summary())

Summary of the built model...
Build model...
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 21, 100)           1239400   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                17024     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 1,256,523
Trainable params: 1,256,523
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
num_epochs =10
batch_size = 128
historyLSTM = modelLSTM.fit(X_train_pad, y_trainDL,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=2,
                    validation_data = [X_val_pad, y_valDL])

Epoch 1/10
76/76 - 13s - loss: 0.8752 - accuracy: 0.6630 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
76/76 - 10s - loss: 0.5075 - accuracy: 0.7989 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
76/76 - 9s - loss: 0.2675 - accuracy: 0.9044 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
76/76 - 9s - loss: 0.1754 - accuracy: 0.9416 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
76/76 - 9s - loss: 0.1270 - accuracy: 0.9563 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
76/76 - 9s - loss: 0.0997 - accuracy: 0.9658 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
76/76 - 9s - loss: 0.0838 - accuracy: 0.9739 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 8/10
76/76 - 9s - loss: 0.0723 - accuracy: 0.9761 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 9/10
76/76 - 10s - loss: 0.0616 - accuracy: 0.9793 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 10/10
76/76 - 9s - loss: 0.0548 - acc

In [ ]:
scoreLSTM,accLSTN= modelLSTM.evaluate(X_test_pad, y_testDL, 
                          batch_size = batch_size, verbose = 2)

26/26 - 1s - loss: 0.0367 - accuracy: 0.9882


In [ ]:
modelLSTM.save("modelLSTM.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
def predict_DLLSTM(s):
  s = sent_tokenize(s)
  text_labels = encoder.classes_
  X_tokens = tokenizer_obj.texts_to_sequences(s)
  X_pad = pad_sequences(X_tokens, maxlen=max_length, padding='post')
  loaded_model= tf.keras.models.load_model('modelLSTM.h5')
  predictions = model.predict(np.array(X_pad))
  predicted_label = text_labels[np.argmax(predictions[0])]
  if predicted_label == 1.0:
    return ('none')  

  elif predicted_label  == 2.0:
      return('sexism')

  elif predicted_label == 3.0:
      return('racism')

s = input('Enter tweet : ')
print(predict_DL(s))

Enter tweet : i hate muslim
racism


In [ ]:
modelSMOTELSTM = build_modelLSTM(vocab_size=vocab_size, EMBEDDING_DIM=EMBEDDING_DIM, max_length=max_length)
print(modelSMOTELSTM.summary())

Build model...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 21, 100)           1239400   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                17024     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 1,256,523
Trainable params: 1,256,523
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_sample_weight("balanced", np.unique(y_train_labels))
num_epochs =10
batch_size = 128
history_SMOTELSTM = modelSMOTELSTM.fit(X_sm, y_sm,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=2,
                    #class_weight=class_weight,
                    validation_data = [X_val_pad, y_testDL])

Epoch 1/10
117/117 - 18s - loss: 0.9197 - accuracy: 0.5574 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
117/117 - 14s - loss: 0.6091 - accuracy: 0.7610 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
117/117 - 14s - loss: 0.4171 - accuracy: 0.8501 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
117/117 - 14s - loss: 0.3203 - accuracy: 0.8902 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
117/117 - 14s - loss: 0.2620 - accuracy: 0.9151 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
117/117 - 14s - loss: 0.2234 - accuracy: 0.9295 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
117/117 - 14s - loss: 0.1894 - accuracy: 0.9406 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 8/10
117/117 - 14s - loss: 0.1727 - accuracy: 0.9471 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 9/10
117/117 - 14s - loss: 0.1524 - accuracy: 0.9528 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 10/10
117/117

In [ ]:
scoreLSTM1,accLSTM1= modelSMOTELSTM.evaluate(X_test_pad, y_testDL, 
                          batch_size = batch_size, verbose = 2)

26/26 - 1s - loss: 0.0976 - accuracy: 0.9727


In [ ]:
modelLSTM.save("modelSMOTELSTM.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
def predict_DLSMOTELSTM(s):
  s = sent_tokenize(s)
  text_labels = encoder.classes_
  X_tokens = tokenizer_obj.texts_to_sequences(s)
  X_pad = pad_sequences(X_tokens, maxlen=max_length, padding='post')
  loaded_model= tf.keras.models.load_model('modelSMOTELSTM.h5')
  predictions = model.predict(np.array(X_pad))
  predicted_label = text_labels[np.argmax(predictions[0])]
  if predicted_label == 1.0:
    return ('none')  

  elif predicted_label  == 2.0:
      return('sexism')

  elif predicted_label == 3.0:
      return('racism')

s = input('Enter tweet : ')
print(predict_DL(s))

Enter tweet : i hate muslim
racism


# Evaluation


*   Accuracy
*   F-Score



# GUI using Anvil

In [ ]:
from getpass import getpass
uplink_key = getpass('Enter your Uplink key: ')

In [ ]:
!pip install anvil-uplink

In [ ]:
import anvil.server

In [ ]:
anvil.server.connect(uplink_key)

In [ ]:
@anvil.server.callable
def predict_ml_svm_with_smote(sentence):
    sent = sent_tokenize(sentence)
    y = vector_transformer.transform(sent)
 #   y_pred = nb.predict(y)
    loaded_model = pickle.load(open(filenameSVMSMOTE, 'rb'))
    y_pred = loaded_model.predict(y)
    if y_pred == 1:
      return ('none')  

    elif y_pred  == 2:
        return('sexism')

    elif y_pred == 3:
        return('racism')

In [ ]:
# anvil.server.wait_forever()